In [1]:
# Notebook plan:
# May try an extremely small dataset here on tyrosine kinase inhibitors 
# and using named entity recognition technique (token classification) first to map ADRs to ADR-related body systems 
# e.g. liver, kidney, cardiovascular, respiratory... and so on (may be useful for other ADR-(on/off)-target recognitions or knowledge graph formation later)
## aim is to classify each ADR term into specific body systems
# (the NLP model concept may make a bit more sense after I try this first...)
# goal is to show most common ADR-related body systems for tyrosine kinase inhibitors (chosen because they're also structurally similar)
# this may involve the full loop of defining, fine-tuning model, setting up metrics & training loop 
# to be able to use the model on a test set of data in the end

# test data will be other tyrosine kinase inhibitors not in this particular dataset
# e.g. dasatinib, afatinib etc. from https://nzf.org.nz/nzf_4653?searchterm=tyrosine%20protein%20and%20tyrosine%20kinase%20inhibitors


In [2]:
import pandas as pd
df = pd.read_csv("Data/cyp_substrates_adrs.csv", delimiter=",")
df

,generic_drug_name,notes,cyp_type_and_cyp_strength_of_evidence,drug_class,adverse_drug_reactions,first_ref,second_ref,date_checked
0,bosentan,NaN,s_3A4,agents_for_pulmonary_hypertension,"abnormal_LFT^^, headache^^, RTI^^, decreased_h...",drugs.com,nzf,270225
1,carbamazepine,NaN,s_3A4,antiepileptics,"constipation^^, leukopenia^^, dizziness^^, dro...",drugs.com,nzf,211024
2,eliglustat,NaN,s_3A4_s_2D6,metabolic_agents,"diarrhea^^, oropharyngeal_pain^^, arthralgia^^...",drugs.com,emc,151124
3,flibanserin,NaN,s_3A4,CNS_agents,"dizziness^^, drowsiness^^, fatigue^, vertigo^,...",drugs.com,Drugs@FDA,161124
4,imatinib,NaN,s_3A4,tyrosine_kinase_inhibitor,"rash^^, diarrhea^^, abdominal_pain^^, constipa...",drugs.com,nzf,181124
...,...,...,...,...,...,...,...,...
79,enzalutamide,*,m_2C8,anti_androgens,"headache^^, dizziness^^, flushing^^, periphera...",drugs.com,nzf,10425
80,repaglinide,NaN,s_2C8,meglinitides,"hypoglycemia^^, headache^^, upper_RTI^^, chest...",drugs.com,Drugs@FDA,10425
81,rosiglitazone,*,m_2C8,thiazolidinediones,"edema^, hypertension^, anemia^, hepatitis(pm),...",drugs.com,Drugs@FDA,20425
82,selexipag,NaN,m_2C8,agents_for_pulmonary_hypertension,"diarrhea^^, flushing^^, sinus_tachycardia^^, r...",drugs.com,nzf,30425


In [3]:
df_tk = df.query("drug_class == 'tyrosine_kinase_inhibitor'")
df_tk

,generic_drug_name,notes,cyp_type_and_cyp_strength_of_evidence,drug_class,adverse_drug_reactions,first_ref,second_ref,date_checked
4,imatinib,NaN,s_3A4,tyrosine_kinase_inhibitor,"rash^^, diarrhea^^, abdominal_pain^^, constipa...",drugs.com,nzf,181124
5,ibrutinib,NaN,s_3A4,tyrosine_kinase_inhibitor,"hypertension^^, AF^^, sinus_tachycardia^^, ra...",drugs.com,nzf,191124
6,neratinib,NaN,s_3A4,tyrosine_kinase_inhibitor,"diarrhea^^, abdominal_pain^^, stomatitis^^, dy...",drugs.com,nzf,201124
58,ruxolitinib,NaN,m_2C9,tyrosine_kinase_inhibitor,"UTI^^, anemia^^, thrombocytopenia^^, neutropen...",drugs.com,nzf,120325
83,tucatinib,*,m_2C8,tyrosine_kinase_inhibitor,"palmar_plantar_erythrodysesthesia_syndrome^^, ...",drugs.com,Drugs@FDA,30425


In [4]:
# Consider using BERT-like models e.g. PubMedBERT, BioBERT for NER in ADRs 
# (core idea is to pre-train and fine-tune transformer models)

In [5]:
tk_adrs = list(df_tk["adverse_drug_reactions"])
tk_adrs

['rash^^, diarrhea^^, abdominal_pain^^, constipation^^, dyspepsia^^, hemorrhage^^, neutropenia^^, thrombocytopenia^^, anemia^^, flu_syndrome^^, weight_gain^^, muscle_spasm/cramps^^, musculoskeletal_pain^^, joint_pain^^, myalgia^^, bone_pain^^, headache^^, dizziness^^, periorbital_edema^^, edema^^, fatigue^^, fever^^, insomnia^^, depression^^, nasopharyngitis^^, cough^^, upper_respiratory_tract infection^^, pharyngolaryngeal_pain^^, sinusitis^^, flushing^, pruritus^, face_edema^, dry skin^, erythema^, alopecia^, night_sweats^, photosensitivity_reaction^, abdominal_distension^, GERD^, dry_mouth^, gastritis^, pancytopenia^, febrile_neutropenia^, increased_LFT^, anorexia^, weight_loss^, joint_swelling^, paresthesia^, hypoesthesia^, eyelid_edema^, lacrimation_increased^, conjunctival_hemorrhage^, conjunctivitis^, dry_eye^, blurred_vision^, taste_disturbance^, weakness^, generalized_edema^, rigors^, dyspnea^, epistaxis^',
 'hypertension^^, AF^^, sinus_tachycardia^^,  rash^^, skin_infection^^

In [6]:
## annotate data for NER
# below adapted function code is part of a class code to produce a NER dataset maker 
# from https://gist.github.com/jangedoo/7ac6fdc7deadc87fd1a1124c9d4ccce9 

# create a tuple of entity values and entity names to prepare data
import re

def assign_tokens_with_entities(texts: str):

    # split the text by spaces only if the space does not occur between square brackets
    # not splitting "multi-word" entity value yet
    raw_tokens = re.split(r"\s(?![^\[]*\])", texts)
    
    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags = re.I | re.M)
    tokens_with_entities = []
    
    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")

            # prefix the name of entity 
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            # split strings by commas (also work for semi-colons or spaces)
            for i, raw_entity_token in enumerate(re.split(r"[,\s;]+", raw_entity_value)):
                entity_prefix = "B" if i == 0 else "I"
                entity_name = f"{entity_prefix}-{raw_entity_name}"
                tokens_with_entities.append((raw_entity_token, entity_name))
        else:
            tokens_with_entities.append((raw_tokens, "O"))
        
    return tokens_with_entities

In [ ]:
## Rest of tk inhibitors ADR texts 
# (in this order: ibrutinib, neratinib, ruxolitinib and tucatinib)
# note: imatinib's ADRs being used as training texts for now; may need to increase size of training texts later

# 'hypertension^^, AF^^, sinus_tachycardia^^,  rash^^, skin_infection^^, pruritus^^, diarrhea^^, stomatitis^^, abdominal_pain^^, constipation^^, dyspepsia^^, GERD^^, UTI^^, decreased_platelets^^, neutropenia^^, decreased_neutrophils^^, decreased_hemoglobin^^, thrombocytopenia^^, hemorrhage^^, anemia^^, lymphocytosis^^, increased_bilirubin^^, increased_AST^^, increased_ALT^^, hepatic_failure(pm), hepatic_cirrhosis(pm), hepatitis_B_reactivation(pm), hepatotoxicity(pm), hypogammaglobulinemia^^, hyperuricemia^^, decreased_appetite^^, hypokalemia^^, musculoskeletal_pain^^, arthralgia^^, muscle_spasms^^, osteonecrosis^^, headache^^, dizziness^^, peripheral_neuropathy(pm)^^, dry_eye^^, blurred_vision^^, increased_lacrimation^^, reduced_visual_acuity^^, conjunctivitis^^, eye_hemorrhage(pm), second_malignancies^^, fatigue^^, fever^^, peripheral_edema^^, pain^^, infections^^, fall^^, sepsis^^, weight_loss^^, insomnia^^, increased_blood_creatinine^^, upper_RTI^^, cough^^, pneumonia^^, dyspnea^^, sinusitis^^, oropharyngeal_pain^^, bronchitis^^, nasopharyngitis^^, flu_syndrome^^, viral_upper_RTI^^, atrial_flutter^, cardiac_failure(pm)^, ventricular_tachyarrhythmias(pm)^, deaths_due_to_cardiac_causes^, erythema(pm)^, urticaria(pm)^, angioedema(pm), Stevens_Johnson_syndrome(pm), onychoclasis(pm), panniculitis(pm), neutrophilic_dermatoses(pm), major_hemorrhage^, febrile_neutropenia^, leukocytosis^,  anaphylactic_shock(pm), tumor_lysis_syndrome(pm)^, hyponatremia^, ischemic_cerebrovascular_events(pm)^, non_melanoma_skin_cancer^, non_skin_carcinomas^, basal_and_squamous_cell_carcinomas^, epistaxis^, interstitial_lung_disease(pm)', /
# 'diarrhea^^, abdominal_pain^^, stomatitis^^, dyspepsia^^, fatigue^^, rash^^, decreased_appetite^^, muscle_spasms^^, abdominal_distention^, dry_mouth^, nail_disorder^, skin_fissures^, weight_loss^, dehydration^, UTI^, increased_ALT^, increased_AST^, epistaxis^', /
# 'UTI^^, anemia^^, thrombocytopenia^^, neutropenia^^, hypercholesterolemia^^, weight_gain^^, dizziness^^, headache^^, bruising^^, bleeding^^, increased_ALT^^, increased_AST^^, GI_bleeding^, tuberculosis^, intracranial_bleeding^, herpes_zoster^', /
# 'palmar_plantar_erythrodysesthesia_syndrome^^, rash^^, diarrhea^^, stomatitis^^, decreased_hemoglobin^^, decreased_phosphate^^, anemia^^, increased_bilirubin^^, increased_ALT^^, increased_AST^^, hepatotoxicity^^, anorexia^^, weight_loss^^, decreased_magnesium^^, decreased_potassium^^, increased_ALP^^, arthralgia^^, increased_creatinine^^, epistaxis^^'

In [8]:
# trialling to assign tokens with entities using a few ADR terms only
assign_tokens_with_entities('[rash](dermatology)^^, [diarrhea](gastrointestinal)^^, [abdominal_pain](gastrointestinal)^^, '
'[constipation](gastrointestinal)^^, [dyspepsia](gastrointestinal)^^, [hemorrhage](haematologic)^^')

# Note: not taking into account of the frequency of occurrences yet (so they're removed for now)

[('rash', 'B-dermatology'),
 ('diarrhea', 'B-gastrointestinal'),
 ('abdominal_pain', 'B-gastrointestinal'),
 ('constipation', 'B-gastrointestinal'),
 ('dyspepsia', 'B-gastrointestinal'),
 ('hemorrhage', 'B-haematologic')]

In [ ]:
# other alternative way is also available re. using tokenize and align labels function after using HF transformers
# e.g. https://huggingface.co/docs/transformers/tasks/token_classification#preprocess
# i.e. using the AutoTokenizer as shown in notebook 5-2
# however, the downside is that this may run in sub-word tokenizations 
# as the tokenizer models are pre-trained so may not align with the specific types of texts we use 
# so some words may be randomly splitted into sub-words

In [ ]:
## Commonly affected body systems in ADR
# nervous system
# psychiatric
 
# ocular
# cardiovascular
# respiratory
# gastrointestinal
# musculoskeletal
# haematologic
# hepatic
# renal
# genitourinary

# metabolic
# endocrine

# dermatologic
# immunologic
# hypersensitivity

# oncologic (carcinogenicity)

# local
# other

In [ ]:
# Preparing imatinib's ADR terms as raw training texts 
# (done manually first, for future use, likely need to look into other quicker way to add labels especially for longer/larger texts)

raw_texts = """
[rash](dermatology)^^, [diarrhea](gastrointestinal)^^, [abdominal_pain](gastrointestinal)^^, 
[constipation](gastrointestinal)^^, [dyspepsia](gastrointestinal)^^, [hemorrhage](haematologic)^^, 
[neutropenia](haematologic)^^, [thrombocytopenia](haematologic)^^, [anemia](haematologic)^^, [flu_syndrome](respiratory)^^, 
[weight_gain](metabolic)^^, [muscle_spasm/cramps](musculoskeletal)^^, [musculoskeletal_pain](musculoskeletal)^^, 
[joint_pain]((musculoskeletal)^^, [myalgia](musculoskeletal)^^, [bone_pain](musculoskeletal)^^, [headache](nervous system)^^, 
[dizziness](nervous system)^^, [periorbital_edema](nervous system)^^, [edema](other)^^, [fatigue](other)^^, 
[fever](other)^^, [insomnia](psychiatric)^^, [depression](psychiatric)^^, [nasopharyngitis](respiratory)^^, [cough](respiratory)^^, 
[upper_respiratory_tract_infection](respiratory)^^, [pharyngolaryngeal_pain](respiratory)^^, [sinusitis](respiratory)^^, 
[flushing](cardiovascular)^, [pruritus](dermatologic)^, [face_edema](hypersensitivity)^, [dry_skin](dermatologic)^, 
[erythema](dermatologic)^, [alopecia](dermatologic)^, [night_sweats](dermatologic)^, [photosensitivity_reaction](dermatologic)^, 
[abdominal_distension](gastrointestinal)^, [GERD](gastrointestinal)^, [dry_mouth](gastrointestinal)^, 
[gastritis](gastrointestinal)^, [pancytopenia](haematologic)^, [febrile_neutropenia](haematologic)^, [increased_LFT](hepatic)^, 
[anorexia](metabolic)^, [weight_loss](metabolic)^, [joint_swelling](musculoskeletal)^, [paresthesia](nervous system)^, 
[hypoesthesia](nervous system)^, [eyelid_edema](ocular)^, [lacrimation_increased](ocular)^, [conjunctival_hemorrhage](ocular)^, 
[conjunctivitis](ocular)^, [dry_eye](ocular)^, [blurred_vision](ocular)^, [taste_disturbance](other)^, [weakness](other)^, 
[generalized_edema](other)^, [rigors](other)^, [dyspnea](respiratory)^, [epistaxis](respiratory)^
"""